# Decision-Tree

## Data Preprocessing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
import random
import counting_fns as cf


# List of all the months
months = ['2_June', '3_July', '4_August', '5_September', '6_October']

# Define the path to the data directory and columns to keep
data_path = "/Users/mau/Library/CloudStorage/Dropbox/Mac/Documents/Dissertation/Chapter 2/Entire_Data/By month"

# Filter Columns
filter = ['session_time', 'gender', 'age_gen', 'first_outcome',
        'first_wager','first_p/b', 'last_outcome', 'last_wager', 'last_p/b',
        'beginning_amt', 'ending_amt', 'ending_balance', 'ave_slotdenom', 
        'std_slotdenom', 'min_slotdenom', 'max_slotdenom', 'ave_theo_payback',
        'min_theo_payback', 'max_theo_payback', 'ave_wageramt', 'std_wageramt',
        'min_wager', 'max_wager', 'ave_p/b', 'std_p/b', 'max_p/b', 'max_profit', 'depletion_slope', 
        '#inc_slotdenom', '#dec_slotdenom', '#inc_maxbet', '#dec_maxbet', '#W', '#L', '#NH', '#D',
        'w/min', 'l/min', '#2ws', '2ws_profit', '2ws_wgramt','2ws/min', 
        '#3ws', '3ws_profit', '3ws_wgramt', '3ws/min', '#4ws', '4ws_profit', '4ws_wgramt', '4ws/min', 
        'w/g', 'l/g', 'nh/g', 'd/g', 'ave_time_per_gamble', 
        'min_time_per_gamble', 'max_time_per_gamble', 'total_gambles',
        'machines_changes', 'unique_machines', 'ave_time_per_machine', 'classification']


## Randomized Months

In [11]:
# Randomly select 3 months for training set
# random.seed(350)
months_train = random.sample(months, 4)

# Print the months in the training set
print("Months in training set:", months_train)

# Create a list of remaining months for the test set
months_test = [month for month in months if month not in months_train]
# Print the months in the test set
print("Months in test set:", months_test)

Months in training set: ['6_October', '2_June', '5_September', '3_July']
Months in test set: ['4_August']


## Load Dataframes

### 1 MIN

In [24]:
# Load the datasets
datasets = cf.load_and_preprocess_datasets_min_ntop(months, data_path, '1min', filter)

# Create training and test sets
dt_train = pd.concat([datasets['dtf'+month[1:]] for month in months_train])
dt_test = pd.concat([datasets['dtf'+month[1:]] for month in months_test])

# # Seperate dependent and independent variables
X_train = dt_train.iloc[:, :-1].values
y_train = dt_train.iloc[:, -1].values
X_test = dt_test.iloc[:, :-1].values
y_test = dt_test.iloc[:, -1].values

# Econde gender column (Binary)
le = LabelEncoder()

# Binary Encode gender
X_train[:, 0] = le.fit_transform(X_train[:, 0])
X_test[:, 0] = le.fit_transform(X_test[:, 0])


# # Encode age_generartion, first_outoce, last_outcome columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 5])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# # Feature Scaling
sc = StandardScaler()

# Scale all columns except the encoded ones
X_train[:, 14:] = sc.fit_transform(X_train[:, 14:])
X_test[:, 14:] = sc.transform(X_test[:, 14:])

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1 Score: ', f1_score(y_test, y_pred, average='macro'))

[[9342 1077]
 [ 962  214]]
Accuracy:  0.8241483398016386
Precision:  0.5362005865980919
Recall:  0.5393019718685055
F1 Score:  0.5375484895530034


In [25]:
from sklearn.inspection import permutation_importance

result = permutation_importance(classifier, X_train, y_train, n_repeats=10, random_state=42)

# Get feature importances and feature names
importances = result.importances_mean
feature_names = ct.get_feature_names_out()

# Sort feature importances
feature_importance = list(zip(feature_names, importances))
feature_importance.sort(key=lambda x: x[1], reverse=True)

# Print feature importances
for feature, importance in feature_importance:
    print(f"{feature}: {importance}")

remainder__x9: 0.10076928336980306
remainder__x8: 0.09980511487964987
remainder__x17: 0.09688183807439824
remainder__x53: 0.09017710612691464
remainder__x26: 0.07200150437636763
encoder__x1_Millenials: 0.06378213894967177
remainder__x10: 0.05090604485776805
remainder__x56: 0.04992136214442015
remainder__x55: 0.0434046772428884
remainder__x36: 0.04079253282275711
remainder__x23: 0.03643667943107219
remainder__x16: 0.03588621444201312
remainder__x35: 0.02969775711159739
remainder__x18: 0.029205415754923414
remainder__x59: 0.026911241794310724
remainder__x24: 0.024842724288840268
remainder__x15: 0.02478118161925602
remainder__x50: 0.024593134573304153
remainder__x22: 0.024350382932166292
remainder__x25: 0.024196526258205696
remainder__x49: 0.023888812910284475
remainder__x3: 0.022165618161925617
remainder__x54: 0.02100998358862145
remainder__x20: 0.020965536105032823
remainder__x7: 0.013563320568927794
remainder__x4: 0.013423140043763682
remainder__x51: 0.013354759299781183
encoder__x1_Ge

## 2 Min

In [13]:
# Load the datasets for 2min, named it datasets_2
datasets_2 = cf.load_and_preprocess_datasets_min_ntop(months, data_path, '2min', filter)

# Create training and test sets
dt_train = pd.concat([datasets_2['dtf'+month[1:]] for month in months_train])
dt_test = pd.concat([datasets_2['dtf'+month[1:]] for month in months_test])

# # Seperate dependent and independent variables
X_train = dt_train.iloc[:, :-1].values
y_train = dt_train.iloc[:, -1].values
X_test = dt_test.iloc[:, :-1].values
y_test = dt_test.iloc[:, -1].values

# Econde gender column (Binary)
le = LabelEncoder()

# Binary Encode gender
X_train[:, 0] = le.fit_transform(X_train[:, 0])
X_test[:, 0] = le.fit_transform(X_test[:, 0])


# # Encode age_generartion, first_outoce, last_outcome columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 5])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# # Feature Scaling
sc = StandardScaler()

# Scale all columns except the encoded ones
X_train[:, 14:] = sc.fit_transform(X_train[:, 14:])
X_test[:, 14:] = sc.transform(X_test[:, 14:])

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1 Score: ', f1_score(y_test, y_pred, average='macro'))

[[9334 1085]
 [ 963  213]]
Accuracy:  0.8233721431651574
Precision:  0.5352881140452146
Recall:  0.538492887797215
F1 Score:  0.5366650001100458


### 3 Min

In [15]:
# Load the datasets for 3min, named it datasets_3
datasets_3 = cf.load_and_preprocess_datasets_min_ntop(months, data_path, '3min', filter)

# Create training and test sets
dt_train = pd.concat([datasets_3['dtf'+month[1:]] for month in months_train])
dt_test = pd.concat([datasets_3['dtf'+month[1:]] for month in months_test])

# # Seperate dependent and independent variables
X_train = dt_train.iloc[:, :-1].values
y_train = dt_train.iloc[:, -1].values
X_test = dt_test.iloc[:, :-1].values
y_test = dt_test.iloc[:, -1].values

# Econde gender column (Binary)
le = LabelEncoder()

# Binary Encode gender
X_train[:, 0] = le.fit_transform(X_train[:, 0])
X_test[:, 0] = le.fit_transform(X_test[:, 0])


# # Encode age_generartion, first_outoce, last_outcome columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 5])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# # Feature Scaling
sc = StandardScaler()

# Scale all columns except the encoded ones
X_train[:, 14:] = sc.fit_transform(X_train[:, 14:])
X_test[:, 14:] = sc.transform(X_test[:, 14:])

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1 Score: ', f1_score(y_test, y_pred, average='macro'))

[[9403 1016]
 [ 973  203]]
Accuracy:  0.8284605433376455
Precision:  0.5363779242563296
Recall:  0.5375524453950886
F1 Score:  0.5369359203397718


### 4 Min

In [16]:
# Load the datasets for 4min, named it datasets_4
datasets_4 = cf.load_and_preprocess_datasets_min_ntop(months, data_path, '4min', filter)

# Create training and test sets
dt_train = pd.concat([datasets_4['dtf'+month[1:]] for month in months_train])
dt_test = pd.concat([datasets_4['dtf'+month[1:]] for month in months_test])

# # Seperate dependent and independent variables
X_train = dt_train.iloc[:, :-1].values
y_train = dt_train.iloc[:, -1].values
X_test = dt_test.iloc[:, :-1].values
y_test = dt_test.iloc[:, -1].values

# Econde gender column (Binary)
le = LabelEncoder()

# Binary Encode gender
X_train[:, 0] = le.fit_transform(X_train[:, 0])
X_test[:, 0] = le.fit_transform(X_test[:, 0])


# # Encode age_generartion, first_outoce, last_outcome columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 5])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# # Feature Scaling
sc = StandardScaler()

# Scale all columns except the encoded ones
X_train[:, 14:] = sc.fit_transform(X_train[:, 14:])
X_test[:, 14:] = sc.transform(X_test[:, 14:])

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1 Score: ', f1_score(y_test, y_pred, average='macro'))

[[9348 1071]
 [ 970  206]]
Accuracy:  0.8239758516601984
Precision:  0.5336525581268975
Recall:  0.5361885468267353
F1 Score:  0.5347672472408462


### 5 MIN

In [17]:
# Load the datasets for 5min, named it datasets_5
datasets_5 = cf.load_and_preprocess_datasets_min_ntop(months, data_path, '5min', filter)

# Create training and test sets
dt_train = pd.concat([datasets_5['dtf'+month[1:]] for month in months_train])
dt_test = pd.concat([datasets_5['dtf'+month[1:]] for month in months_test])

# # Seperate dependent and independent variables
X_train = dt_train.iloc[:, :-1].values
y_train = dt_train.iloc[:, -1].values
X_test = dt_test.iloc[:, :-1].values
y_test = dt_test.iloc[:, -1].values

# Econde gender column (Binary)
le = LabelEncoder()

# Binary Encode gender
X_train[:, 0] = le.fit_transform(X_train[:, 0])
X_test[:, 0] = le.fit_transform(X_test[:, 0])


# # Encode age_generartion, first_outoce, last_outcome columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 5])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# # Feature Scaling
sc = StandardScaler()

# Scale all columns except the encoded ones
X_train[:, 14:] = sc.fit_transform(X_train[:, 14:])
X_test[:, 14:] = sc.transform(X_test[:, 14:])

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1 Score: ', f1_score(y_test, y_pred, average='macro'))

[[9409 1010]
 [ 955  221]]
Accuracy:  0.8305304010349288
Precision:  0.5436914743624512
Recall:  0.5454934421220259
F1 Score:  0.544541323511888


### 10 MIN

In [18]:
# Load the datasets for 5min, named it datasets_5
datasets_10 = cf.load_and_preprocess_datasets_min_ntop(months, data_path, '10min', filter)

# Create training and test sets
dt_train = pd.concat([datasets_5['dtf'+month[1:]] for month in months_train])
dt_test = pd.concat([datasets_5['dtf'+month[1:]] for month in months_test])

# # Seperate dependent and independent variables
X_train = dt_train.iloc[:, :-1].values
y_train = dt_train.iloc[:, -1].values
X_test = dt_test.iloc[:, :-1].values
y_test = dt_test.iloc[:, -1].values

# Econde gender column (Binary)
le = LabelEncoder()

# Binary Encode gender
X_train[:, 0] = le.fit_transform(X_train[:, 0])
X_test[:, 0] = le.fit_transform(X_test[:, 0])


# # Encode age_generartion, first_outoce, last_outcome columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 5])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# # Feature Scaling
sc = StandardScaler()

# Scale all columns except the encoded ones
X_train[:, 14:] = sc.fit_transform(X_train[:, 14:])
X_test[:, 14:] = sc.transform(X_test[:, 14:])

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1 Score: ', f1_score(y_test, y_pred, average='macro'))

[[9409 1010]
 [ 955  221]]
Accuracy:  0.8305304010349288
Precision:  0.5436914743624512
Recall:  0.5454934421220259
F1 Score:  0.544541323511888


### 15 MIN

In [27]:
# Load the datasets for 5min, named it datasets_5
datasets_15 = cf.load_and_preprocess_datasets_min_ntop(months, data_path, '10min', filter)

# Create training and test sets
dt_train = pd.concat([datasets_5['dtf'+month[1:]] for month in months_train])
dt_test = pd.concat([datasets_5['dtf'+month[1:]] for month in months_test])

# # Seperate dependent and independent variables
X_train = dt_train.iloc[:, :-1].values
y_train = dt_train.iloc[:, -1].values
X_test = dt_test.iloc[:, :-1].values
y_test = dt_test.iloc[:, -1].values

# Econde gender column (Binary)
le = LabelEncoder()

# Binary Encode gender
X_train[:, 0] = le.fit_transform(X_train[:, 0])
X_test[:, 0] = le.fit_transform(X_test[:, 0])


# # Encode age_generartion, first_outoce, last_outcome columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1, 2, 5])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# # Feature Scaling
sc = StandardScaler()

# Scale all columns except the encoded ones
X_train[:, 14:] = sc.fit_transform(X_train[:, 14:])
X_test[:, 14:] = sc.transform(X_test[:, 14:])

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1 Score: ', f1_score(y_test, y_pred, average='macro'))

[[9409 1010]
 [ 955  221]]
Accuracy:  0.8305304010349288
Precision:  0.5436914743624512
Recall:  0.5454934421220259
F1 Score:  0.544541323511888


In [28]:
from sklearn.inspection import permutation_importance

result = permutation_importance(classifier, X_train, y_train, n_repeats=10, random_state=42)

# Get feature importances and feature names
importances = result.importances_mean
feature_names = ct.get_feature_names_out()

# Sort feature importances
feature_importance = list(zip(feature_names, importances))
feature_importance.sort(key=lambda x: x[1], reverse=True)

# Print feature importances
for feature, importance in feature_importance:
    print(f"{feature}: {importance}")

remainder__x56: 0.13194064551422321
remainder__x9: 0.10338826586433263
remainder__x17: 0.08687431619256018
remainder__x59: 0.056379923413566735
encoder__x1_Millenials: 0.04567491794310723
remainder__x55: 0.03672729759299781
remainder__x22: 0.035462253829321666
remainder__x10: 0.0331578227571116
remainder__x53: 0.03268257658643327
remainder__x50: 0.03217997811816194
remainder__x8: 0.030514907002188184
remainder__x26: 0.029827680525164123
remainder__x23: 0.028901121444201316
remainder__x16: 0.028227571115973737
remainder__x36: 0.026692423413566736
remainder__x25: 0.026076996717724298
remainder__x35: 0.02536925601750547
remainder__x49: 0.022716083150984678
remainder__x32: 0.021560448577680514
remainder__x18: 0.020141548140043763
remainder__x54: 0.019974015317286663
remainder__x24: 0.017710612691466086
remainder__x15: 0.016780634573304164
remainder__x52: 0.015806208971553614
remainder__x20: 0.014510393873085326
remainder__x6: 0.014182166301969367
remainder__x38: 0.014130880743982489
remain